<a href="https://colab.research.google.com/github/luanakwon/GreatBarrierReef/blob/main/GBR_YOLOBasic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download dataset from kaggle api
kaggle  
https://www.kaggle.com/c/tensorflow-great-barrier-reef/data  

kaggle API usage  
https://colab.research.google.com/github/corrieann/kaggle/blob/master/kaggle_api_in_colab.ipynb

In [1]:
# !pip install kaggle
# !pip install --upgrade --force-reinstall --no-deps kaggle

In [2]:
# from google.colab import files
# 
# uploaded = files.upload()
# 
# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))
#   
# Then move kaggle.json into the folder where the API expects to find it.
# !mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [3]:
# !kaggle competitions download -c tensorflow-great-barrier-reef

In [4]:
# !unzip -q /content/tensorflow-great-barrier-reef.zip 

# Session 1
---
Useful links  
- [YOLOv5 - Train Custom Data github](https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data)  


In [5]:
"""
# create hierarchy
import os

for f1 in 'train', 'val', 'test':
  for f2 in 'images', 'labels':
    os.makedirs(f'/content/Datasets/{f1}/{f2}')
    
"""

"\n# create hierarchy\nimport os\n\nfor f1 in 'train', 'val', 'test':\n  for f2 in 'images', 'labels':\n    os.makedirs(f'/content/Datasets/{f1}/{f2}')\n    \n"

In [6]:
"""
# get train.csv
import pandas as pd

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train
"""

"\n# get train.csv\nimport pandas as pd\n\ndf_train = pd.read_csv('train.csv')\ndf_test = pd.read_csv('test.csv')\ndf_train\n"

In [7]:
"""
def annotation2YoloFormat(_annotation, width=1280, height=720):
  out = ''
  if isinstance(_annotation,str):
    a = _annotation
    while True:
      ix = a.find("'x': ")
      iy = a.find("'y': ")
      iw = a.find("'width': ")
      ih = a.find("'height': ")
      
      if ix < 0 or iy < 0 or iw < 0 or ih < 0:
        break

      x = float(a[ix + len("'x': "):a.find(", 'y")])/width
      y = float(a[iy + len("'y': "):a.find(", 'w")])/height
      w = float(a[iw + len("'width': "):a.find(", 'h")])/width
      h = float(a[ih + len("'height': "):a.find('}')])/height
      
      
      out += '0 %.6f %.6f %.6f %.6f\n'%(x+w/2, y+h/2, w, h)
      a = a[a.find('}')+1:]
    
    out = out[:-1]
  return out
  """

'\ndef annotation2YoloFormat(_annotation, width=1280, height=720):\n  out = \'\'\n  if isinstance(_annotation,str):\n    a = _annotation\n    while True:\n      ix = a.find("\'x\': ")\n      iy = a.find("\'y\': ")\n      iw = a.find("\'width\': ")\n      ih = a.find("\'height\': ")\n      \n      if ix < 0 or iy < 0 or iw < 0 or ih < 0:\n        break\n\n      x = float(a[ix + len("\'x\': "):a.find(", \'y")])/width\n      y = float(a[iy + len("\'y\': "):a.find(", \'w")])/height\n      w = float(a[iw + len("\'width\': "):a.find(", \'h")])/width\n      h = float(a[ih + len("\'height\': "):a.find(\'}\')])/height\n      \n      \n      out += \'0 %.6f %.6f %.6f %.6f\n\'%(x+w/2, y+h/2, w, h)\n      a = a[a.find(\'}\')+1:]\n    \n    out = out[:-1]\n  return out\n  '

In [8]:
"""
# read images from /content/train_images
# change name
# save the image to train | val | test. ratio = 0.64 : 0.16 : 0.2
# create corresponding txt file and save

# change image name to image id
# split images by train:val:test = .64:.16:.2
import shutil
import os
import random
random.seed(31415)
%cd train_images

TEST_DIRECTORY = "../Datasets/test"
TRAIN_DIRECTORY = "../Datasets/train"
VAL_DIRECTORY = "../Datasets/val"

for dir in os.listdir():
  # print(dir)
  for image in os.listdir(dir):
    annot = df_train.loc[df_train['image_id']==f'{dir[-1]}-{image[:-4]}', 'annotations'].iloc[0]
    if annot != '[]':
      random_key = random.random()
      if random_key < 0.64:
        random_folder = TRAIN_DIRECTORY
      elif random_key < 0.8:
        random_folder = VAL_DIRECTORY
      else:
        random_folder = TEST_DIRECTORY
      shutil.move(f'{dir}/{image}', f"../Datasets/{random_folder}/images/{dir[-1]}-{image}")

      # might add label as well while in iteration
      # as YOLO format [{xywh},{},...]
      f = open(f"../Datasets/{random_folder}/labels/{dir[-1]}-{image[:-4]}.txt", 'w')
      f.write(annotation2YoloFormat(annot))
      f.close()

%cd ..
!rm -rf train_images/
"""

'\n# read images from /content/train_images\n# change name\n# save the image to train | val | test. ratio = 0.64 : 0.16 : 0.2\n# create corresponding txt file and save\n\n# change image name to image id\n# split images by train:val:test = .64:.16:.2\nimport shutil\nimport os\nimport random\nrandom.seed(31415)\n%cd train_images\n\nTEST_DIRECTORY = "../Datasets/test"\nTRAIN_DIRECTORY = "../Datasets/train"\nVAL_DIRECTORY = "../Datasets/val"\n\nfor dir in os.listdir():\n  # print(dir)\n  for image in os.listdir(dir):\n    annot = df_train.loc[df_train[\'image_id\']==f\'{dir[-1]}-{image[:-4]}\', \'annotations\'].iloc[0]\n    if annot != \'[]\':\n      random_key = random.random()\n      if random_key < 0.64:\n        random_folder = TRAIN_DIRECTORY\n      elif random_key < 0.8:\n        random_folder = VAL_DIRECTORY\n      else:\n        random_folder = TEST_DIRECTORY\n      shutil.move(f\'{dir}/{image}\', f"../Datasets/{random_folder}/images/{dir[-1]}-{image}")\n\n      # might add label a

In [9]:
"""
#!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -r requirements.txt  # install

import torch
from yolov5 import utils
# display = utils.notebook_init()  # checks
"""

'\n#!git clone https://github.com/ultralytics/yolov5  # clone\n%cd yolov5\n%pip install -r requirements.txt  # install\n\nimport torch\nfrom yolov5 import utils\n# display = utils.notebook_init()  # checks\n'

# Under /content/yolov5

In [10]:
"""
# create yaml file and save to /data (your at /content/yolov5)
# https://drive.google.com/file/d/1BBrQ5_1urHvJdYOmYPR-HalMeF9dnK37/view?usp=sharing
!gdown --id 1BBrQ5_1urHvJdYOmYPR-HalMeF9dnK37 -O /content/yolov5/data/GBR.yaml
"""

'\n# create yaml file and save to /data (your at /content/yolov5)\n# https://drive.google.com/file/d/1BBrQ5_1urHvJdYOmYPR-HalMeF9dnK37/view?usp=sharing\n!gdown --id 1BBrQ5_1urHvJdYOmYPR-HalMeF9dnK37 -O /content/yolov5/data/GBR.yaml\n'

In [11]:
# Train YOLOv5s on COCO128 for 3 epochs
# --img : image size (size,size)
# --batch : batch size
# --epochs : epochs
# --data : .yaml file name
# --weights : pretrained weights ('' --cfg yolov5s.yaml if from scratch)
# !python train.py --img 720 --batch 20 --epochs 12 --data GBR.yaml --weights yolov5m.pt

# Visualization

In [12]:
"""
%cd /content/yolov5
!python detect.py --img 640\
 --source ../Datasets/test/images\
 --weights runs/train/exp2/weights/best.pt\
 --data data/GBR.yaml\
 --nosave\
 --save-txt\
 --save-conf\
 --conf-thres 0.01\
 --iou-thres 0.3\
 --max-det 100
"""

'\n%cd /content/yolov5\n!python detect.py --img 640 --source ../Datasets/test/images --weights runs/train/exp2/weights/best.pt --data data/GBR.yaml --nosave --save-txt --save-conf --conf-thres 0.01 --iou-thres 0.3 --max-det 100\n'

In [13]:
"""
%cd /content/yolov5/runs/detect/exp2/labels
!zip /content/exp_pred_labels.zip *.txt
%cd /content/Datasets/test/labels
!zip /content/exp_true_labels.zip *.txt
%cd /content
"""

'\n%cd /content/yolov5/runs/detect/exp2/labels\n!zip /content/exp_pred_labels.zip *.txt\n%cd /content/Datasets/test/labels\n!zip /content/exp_true_labels.zip *.txt\n%cd /content\n'

In [14]:
"""
%cd /content/yolov5
# visualize
from utils.plots import plot_results 
plot_results('./runs/train/exp2/results.csv')  # plot 'results.csv' as 'results.png'
"""

"\n%cd /content/yolov5\n# visualize\nfrom utils.plots import plot_results \nplot_results('./runs/train/exp2/results.csv')  # plot 'results.csv' as 'results.png'\n"

In [15]:
"""
import cv2
from google.colab.patches import cv2_imshow
cv2_imshow(cv2.imread('./runs/train/exp2/results.png'))
"""

"\nimport cv2\nfrom google.colab.patches import cv2_imshow\ncv2_imshow(cv2.imread('./runs/train/exp2/results.png'))\n"

# Submit

In [16]:
"""
# https://drive.google.com/file/d/1Koc5DW5obyZ5blVzdP5EHeaFDfmUNkE0/view?usp=sharing
# save custom detect.py to /content/yolov5/detect2.py
!gdown --id 1Koc5DW5obyZ5blVzdP5EHeaFDfmUNkE0 -O /content/yolov5/detect2.py

# https://drive.google.com/file/d/1JMNnFYJOgKZTA1a0RGBMbRO2EFouZU4K/view?usp=sharing
# save custom datasets.py to /content/yolov5/utils/datasets.py
!gdown --id 1JMNnFYJOgKZTA1a0RGBMbRO2EFouZU4K -O /content/yolov5/utils/datasets.py
"""

'\n# https://drive.google.com/file/d/1Koc5DW5obyZ5blVzdP5EHeaFDfmUNkE0/view?usp=sharing\n# save custom detect.py to /content/yolov5/detect2.py\n!gdown --id 1Koc5DW5obyZ5blVzdP5EHeaFDfmUNkE0 -O /content/yolov5/detect2.py\n\n# https://drive.google.com/file/d/1JMNnFYJOgKZTA1a0RGBMbRO2EFouZU4K/view?usp=sharing\n# save custom datasets.py to /content/yolov5/utils/datasets.py\n!gdown --id 1JMNnFYJOgKZTA1a0RGBMbRO2EFouZU4K -O /content/yolov5/utils/datasets.py\n'

In [17]:
%cd ./working

[Errno 2] No such file or directory: './working'
/kaggle/working


In [18]:
# something like this...?
import greatbarrierreef
%cd ../input/yologithubrepoandsomecustom
from yolov5.detect2 import run as detect2run
%cd ../../working
%ls

detect2run(
    weights='../input/yologithubrepoandsomecustom/yolov5/weights/best.pt',
    data='../input/yologithubrepoandsomecustom/yolov5/data/GBR.yaml',
    env=greatbarrierreef.make_env(),
    conf_thres=0.01,
    iou_thres=0.3,
    max_det=100
)

/kaggle/input/yologithubrepoandsomecustom
/kaggle/working
__notebook__.ipynb


YOLOv5 🚀 v6.0-235-ge1a6a0b torch 1.9.1+cpu CPU

Fusing layers... 
Model Summary: 290 layers, 20852934 parameters, 0 gradients


This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


In [19]:
import pandas
submission_df = pandas.read_csv('./submission.csv')
submission_df

,index,annotations
0,0,0.197906 265 255 40 30
1,1,0.011626 261 260 37 28 0.011906 1234 667 45 50
2,2,0.052808 257 260 40 30
